In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error
from nixtla import NixtlaClient
import pandas as pd

In [2]:
API_KEY = 'nixtla-tok-7FJsZBnuXE04TuDFi0UrUmeoV03oHl9FJbe0h7cxDMiyIFuch5RvIcoKGQki5erTO05vHh2TDtFJQpJ1'
nixtla_client = NixtlaClient(api_key=API_KEY)
nixtla_client.validate_api_key()

In [4]:
xlsx_files = [
    "Wind farm site 1 (Nominal capacity-99MW).xlsx",
    "Wind farm site 2 (Nominal capacity-200MW).xlsx",
    "Wind farm site 3 (Nominal capacity-99MW).xlsx",
    "Wind farm site 4 (Nominal capacity-66MW).xlsx",
    "Wind farm site 5 (Nominal capacity-36MW).xlsx",
    "Wind farm site 6 (Nominal capacity-96MW).xlsx"
]

In [5]:
df = pd.read_excel(f'datasets/{xlsx_files[0]}')

In [6]:
nixtla_client.plot(df, time_col='Time(year-month-day h:m:s)', target_col='value')

c:\Users\ardahal\AppData\Local\anaconda3\envs\test\Lib\site-packages\utilsforecast\plotting.py:34: SyntaxWarning: invalid escape sequence '\('
  models_pat = r"|".join(models).replace("(", "\(").replace(")", "\)")
c:\Users\ardahal\AppData\Local\anaconda3\envs\test\Lib\site-packages\utilsforecast\plotting.py:34: SyntaxWarning: invalid escape sequence '\)'
  models_pat = r"|".join(models).replace("(", "\(").replace(")", "\)")


ValueError: The following columns are missing: ['value']